#General Concepts

Artificial intelligence is the field of creating computer systems to be able to perform intelligent human behavior.
Machine Learning is a subset of AI that involves supervised, unsupervised, and reinforcement learning. It invovles the machine learning the correct or optimal solutions to a problem. 
Deep learning is a subset of machine learning where the models are large neural networks. 


#Building a Model

Convolutional neural network are models applied to computer vision problems. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
tr_labels = keras.utils.to_categorical(train_labels)
te_labels = keras.utils.to_categorical(test_labels)
tr_images = train_images/255
te_images = test_images/255

In Keras, you first have to initialize the model. Here we are initializing a sequential model.

In [0]:
model = keras.models.Sequential()

ConvNets are built with multiple layers. The main layers of the ConvNet are the Convolutional Layers. The first convolutional layer is the input layer so you have to pass in the expected input shape. 

Convolutional layers take a kernel size and stride to determine how big the kernel is and how it will move along the input. You need to determine the number of output filters you want. Padding can be added to retain the spatial dimensions of the input. Activation functions are used in these layers to generally add some non-linearity to the model. ReLU is one of the more common activation functions. 

In [0]:
model.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32,3)))
model.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu'))

With more convolutional layers, the number of filters generally increases while the size of the images stays the same if the padding is set to retain the spatial size of the input. This can lead to the networks getting too large in size. This is where max pooling comes in. Max pooling with a size of (2,2) will halve the spatial dimensions. 

In [0]:
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

Conolutional neural networks are deep networks built with many layers. 

In [0]:
model.add(keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

The output of convnets first needs to be flattened to a vector. 

In [0]:
model.add(keras.layers.Flatten())

Once flattened, dense layers, where each neuron receives all the previous input, are used. Like conv layers, an activation function is used.

In [0]:
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))

Dropout is applied to prevent overfitting. It means that a percentage of random neurons will be ignored. 

In [0]:
model.add(keras.layers.Dropout(0.2))

Finally, you need the output layer. The output layer size should be the number classes you have for image classification. The activation function can vary depending on the problem. Generally, for classification of multiple classes, softmax is used.

In [0]:
model.add(keras.layers.Dense(10, activation='softmax'))

#Compiling the Model

Once the model is built, the next step is to compile the model. The first step is to determine the loss function. The type of loss function you use is dependent on the problem you are trying to solve. For this case, since we are doing image classification, the loss is categorical crossentropy. 

The next step is to determine the optimizer you want to use. Optimizers are how the model will update the parameters of your network based on the loss. There are many optimizers that you can use, such as, stochastic gradient descent (SGD), Adam, and RMSProp. The most important parameter to set for optimizers is the learning rate. If you set the learning rate too high, the model may diverge. If you set it too low, the model will take a long time.

In [0]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Training the Model

Once the model is compiled, the next step train the model to the training data. You can loop through the training data as many times as you want while training. One loop is called an epoch. After each epoch you test the current model on validation data to see if the model is improving after each epoch. 

In [0]:
epochs = 10
history = model.fit(tr_images, 
                      tr_labels, 
                      epochs=epochs,  
                      validation_data=(te_images, te_labels))

You have to be careful though. If you train the model for too few epochs, or your model is too simple, you might underfit to the data, meaning that the model didn't learn the data. 

On the opposite side, if you train a model for too long, or your model is too complex, you can overfit the model. An overfitted model means that the model doesn't generalize well to new data.

#Finetuning a Pretrained Model

Training a neural network is time consuming task. To speed up the process it is common to take a network that has already been trained on data and finetune it to your specific data.

In [19]:
vgg = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Here, we take a vgg16 network that has been trained on imagenet. Include top set to false means to only take the hidden layers up to before flattening the input. We will then freeze the layers of vgg16 so they cannot be updated. 

In [0]:
for layer in vgg.layers:
    layer.trainable = False

Now you can add any new layers to the network and run the network.

In [0]:
model = keras.models.Sequential()
model.add(vgg)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

In [25]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________
